In [8]:
import PySpice.Logging.Logging as Logging
logger = Logging.setup_logging()
from PySpice.Spice.Netlist import Circuit
from PySpice.Unit import *

# Define the circuit
circuit = Circuit('Example Circuit 2')
circuit.R(1, 1, 2, 5@u_Ω)
circuit.R(2, 2, circuit.gnd, 10@u_Ω)
circuit.R(3, 2, 3, 20@u_Ω)
circuit.R(4, 3, circuit.gnd, 40@u_Ω)
circuit.V(1, 1, circuit.gnd, 50@u_V)
circuit.I(1, circuit.gnd, 3, 3@u_A)

# Run the simulation
simulator = circuit.simulator(temperature=25, nominal_temperature=25)
analysis = simulator.operating_point()

# Print all branch currents (includes sources and resistors)
print("Branch Currents:")
for branch, current in analysis.branches.items():
    print(f'  {branch}: {float(current[0]):5.3f} A')

# Print node voltages
print("\nNode Voltages:")
for node, voltage in analysis.nodes.items():
    print(f'  {node}: {float(voltage[0]):5.2f} V')

# Calculate and print resistor currents using Ohm's law
print("\nResistor Currents (calculated from V=IR):")
resistors = [circuit.R1, circuit.R2, circuit.R3, circuit.R4]
for resistor in resistors:
    # Get the nodes connected to this resistor
    n1, n2 = resistor.nodes
    
    # Get voltages at both nodes (convert to float)
    v1 = float(analysis.nodes[str(n1)][0]) if str(n1) != '0' else 0.0
    v2 = float(analysis.nodes[str(n2)][0]) if str(n2) != '0' else 0.0
    
    # Calculate voltage across resistor
    v_resistor = v1 - v2
    
    # Calculate current using Ohm's law (I = V/R)
    resistance = float(resistor.resistance)
    current = v_resistor / resistance
    
    print(f'  {resistor.name}: {current:5.3f} A (from {n1} to {n2})')
    print(f'    Voltage across: {v_resistor:5.2f} V, Resistance: {resistance:5.1f} Ω')

AttributeError: add_current_probe